In [95]:
import os
import numpy as np
import pandas as pd
import zipfile
import nltk
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import pickle

In [96]:
import logging

logging.basicConfig(format= '%(asctime)s, %(levelname)s, %(message)s',level = logging.INFO)

In [97]:
target_path = os.path.join(os.path.dirname(os.getcwd()),'data')

In [98]:
test_data = pd.read_csv(os.path.join(target_path, 'testData.tsv'), quoting = 3, delimiter = '\t' )
labeled_data = pd.read_csv(os.path.join(target_path, 'labeledTrainData.tsv'), quoting = 3, delimiter = '\t' )
unlabeled_data = pd.read_csv(os.path.join(target_path, 'unlabeledTrainData.tsv'), quoting = 3, delimiter = '\t' )

In [99]:
## 除了part1给出的BOW方法之外，接下来将使用另外两种文本的特征向量表示来训练模型。范式均为将一则训练样本
# 映射到一个n维的特征空间中
# 第一种是part2中使用的为w2v方法
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10words.model")

2023-05-18 16:26:04,276, INFO, loading Word2Vec object from 300features_40minwords_10words.model
2023-05-18 16:26:04,302, INFO, loading wv recursively from 300features_40minwords_10words.model.wv.* with mmap=None
2023-05-18 16:26:04,302, INFO, setting ignored attribute cum_table to None
2023-05-18 16:26:04,427, INFO, Word2Vec lifecycle event {'fname': '300features_40minwords_10words.model', 'datetime': '2023-05-18T16:26:04.426562', 'gensim': '4.3.0', 'python': '3.9.0 (default, Nov 15 2020, 08:30:55) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'loaded'}


In [100]:
# type(model.syn0) 被弃用力（悲）
print(type(model.wv.vectors))

model.wv.vectors.shape

<class 'numpy.ndarray'>


(16490, 300)

In [101]:
# 数据清洁
def data_proc(review, re_stop=False):
    # 爬虫数据去标签
    re_HTML_data = BeautifulSoup(review).get_text()
    # 去标点符号
    re_punc_data = re.sub('[^a-zA-Z]',' ',re_HTML_data)
    # 统一小写并变成词组
    words = re_punc_data.lower().split()
    # 去除stopword?
    if re_stop:
        stops = set(stopwords.words('English'))
        words = [w for w in words if not w in stops]
    
    return words 

# tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# sentence_list = tokenizer.tokenize(unlabeled_data['review'][0])
# def to_sentences(reivew, tokenizer, re_stop = False):
    
#     sentence_list = tokenizer.tokenize(review)
    
#     sentences = []
    
#     for st in sentence_list:
#         if len(st) > 0:
#             sentences.append(data_proc(st, re_stop))
    
#     return sentences

In [102]:
def review_words_to_single(review, model,num_feature):
    feature_vectors = np.zeros(num_feature, dtype = 'float32')
    
#     feature_vab = set(model.index2word) 又又又被弃用力（悲上悲）
    feature_vab = set(model.wv.index_to_key) # set加速
    
    counter = 0
    
    for word in review:
        if word in feature_vab:
            feature_vectors = np.add(feature_vectors, model.wv.get_vector(word))
            counter += 1
    
    feature_vectors = np.divide(feature_vectors, counter)
    return feature_vectors
    
def gen_feature_data(reviews, model, num_feature):
    feature_data = np.zeros((len(reviews),num_feature), dtype = 'float32' )
    
    counter = 0
    
    for review in reviews:
        if (counter+1) % 1000 == 0:
            print("review %d of %d" % (counter + 1,len(reviews)) )
        feature_data[counter] =  review_words_to_single(review, model, num_feature)
        counter = counter + 1
    
    return feature_data

In [103]:
# 根据生数据，生成分词后的评论列表
clear_train_reviews = []
clear_test_reviews = []
num_feature = 300


# for review in labeled_data['review']:
#     clear_train_reviews.append(data_proc(review, re_stop = True))

# train_data = gen_feature_data(clear_train_reviews, model, num_feature)

# for review in test_data['review']:
#     clear_test_reviews.append(data_proc(review, re_stop = True))

# test_feature_data = gen_feature_data(clear_test_reviews, model, num_feature)


# import pickle

# with open('w2v_train_data.pkl','wb') as f:
#     pickle.dump(train_data,f)
    
# with open('w2v_test_feature_data.pkl','wb') as f:
#     pickle.dump(test_feature_data,f)

with open('w2v_train_data.pkl','rb') as f:
    train_data = pickle.load(f)
    
with open('w2v_test_feature_data.pkl','rb') as f:
    test_feature_data = pickle.load(f)

C:\Users\FlashBlack7\AppData\Local\Temp\ipykernel_17744\2268034714.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  re_HTML_data = BeautifulSoup(review).get_text()


review 1000 of 25000
review 2000 of 25000
review 3000 of 25000
review 4000 of 25000
review 5000 of 25000
review 6000 of 25000
review 7000 of 25000
review 8000 of 25000
review 9000 of 25000
review 10000 of 25000
review 11000 of 25000
review 12000 of 25000
review 13000 of 25000
review 14000 of 25000
review 15000 of 25000
review 16000 of 25000
review 17000 of 25000
review 18000 of 25000
review 19000 of 25000
review 20000 of 25000
review 21000 of 25000
review 22000 of 25000
review 23000 of 25000
review 24000 of 25000
review 25000 of 25000
review 1000 of 25000
review 2000 of 25000
review 3000 of 25000
review 4000 of 25000
review 5000 of 25000
review 6000 of 25000
review 7000 of 25000
review 8000 of 25000
review 9000 of 25000
review 10000 of 25000
review 11000 of 25000
review 12000 of 25000
review 13000 of 25000
review 14000 of 25000
review 15000 of 25000
review 16000 of 25000
review 17000 of 25000
review 18000 of 25000
review 19000 of 25000
review 20000 of 25000
review 21000 of 25000
review

NameError: name 'test_feature_data_' is not defined

In [105]:
import joblib

joblib.dump(forest,'w2v_forest.joblib')

from sklearn.ensemble import RandomForestClassifier

# forest = RandomForestClassifier(n_estimators = 100)

forest = joblib.load('w2v_forest.joblib')

# forest.fit(train_data, labeled_data['sentiment'])

result = forest.predict(test_feature_data)

prediction_sub = pd.DataFrame(data={'id':test_data['id'],'sentiment':result})
prediction_to_seen = pd.DataFrame(data={'review':test_data['review'],'sentiment':result})

prediction_sub.to_csv('w2v_RF_pre_sub.csv',index=False, quoting=3 )
prediction_to_seen.to_csv('w2v_RF_pre.csv',index=False )

In [86]:
## 基于聚类的Bags of Words（方法二）
# 将评论中所有单词的词向量相加并平均损失了很多信息，
# 最后使用随机森林分类的效果甚至低于基于词频统计的Bag of Words，实在是暴殄天物
# 既然类Bags of Words的方法在基于决策树的分离器模型上运行效果较好，那干脆用一种新的范式做Bags of Words好了
from sklearn.cluster import KMeans
# import time

# start = time.time()

# num_clusters = round(model.wv.vectors.shape[0]/5)

# k_means = KMeans(n_clusters = num_clusters, init = 'k-means++',verbose=1)

# idx = k_means.fit_predict(model.wv.vectors)

# end = time.time()
# elapsed = end - start
# print ("Time taken for K Means clustering: ", elapsed, "seconds.")

# joblib.dump(k_means,'w2v_kmeans.joblib')

k_means = joblib.load('w2v_kmeans.joblib')

D:\SoftWare\Anaconda\envs\pytorch\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Initialization complete
Iteration 0, inertia 284449.71875.
Iteration 1, inertia 224825.1875.
Iteration 2, inertia 221281.109375.
Iteration 3, inertia 220102.796875.
Iteration 4, inertia 219624.828125.
Iteration 5, inertia 219334.734375.
Iteration 6, inertia 219125.875.
Iteration 7, inertia 218966.84375.
Iteration 8, inertia 218846.765625.
Iteration 9, inertia 218734.46875.
Iteration 10, inertia 218655.0625.
Iteration 11, inertia 218599.375.
Iteration 12, inertia 218562.03125.
Iteration 13, inertia 218519.34375.
Iteration 14, inertia 218486.734375.
Iteration 15, inertia 218452.75.
Iteration 16, inertia 218408.4375.
Iteration 17, inertia 218380.859375.
Iteration 18, inertia 218348.4375.
Iteration 19, inertia 218333.15625.
Iteration 20, inertia 218319.328125.
Iteration 21, inertia 218298.34375.
Iteration 22, inertia 218291.140625.
Iteration 23, inertia 218282.65625.
Iteration 24, inertia 218274.125.
Iteration 25, inertia 218269.140625.
Iteration 26, inertia 218268.796875.
Iteration 27, in

Iteration 4, inertia 219868.71875.
Iteration 5, inertia 219515.03125.
Iteration 6, inertia 219302.453125.
Iteration 7, inertia 219188.921875.
Iteration 8, inertia 219105.296875.
Iteration 9, inertia 219022.15625.
Iteration 10, inertia 218962.375.
Iteration 11, inertia 218906.921875.
Iteration 12, inertia 218876.75.
Iteration 13, inertia 218853.09375.
Iteration 14, inertia 218828.140625.
Iteration 15, inertia 218811.21875.
Iteration 16, inertia 218802.859375.
Iteration 17, inertia 218791.265625.
Iteration 18, inertia 218784.828125.
Iteration 19, inertia 218777.140625.
Iteration 20, inertia 218763.3125.
Iteration 21, inertia 218760.515625.
Iteration 22, inertia 218758.609375.
Iteration 23, inertia 218758.234375.
Iteration 24, inertia 218758.171875.
Converged at iteration 24: strict convergence.
Initialization complete
Iteration 0, inertia 285664.0625.
Iteration 1, inertia 225336.671875.
Iteration 2, inertia 221569.0.
Iteration 3, inertia 220305.75.
Iteration 4, inertia 219718.1875.
Itera

In [138]:
import pickle

with open("cluster_word_map_idx.pkl","rb") as f:
    word_map_idx = pickle.load(f)
    
# model.wv.index_to_key
# word_map_idx = dict(zip(model.wv.index_to_key, idx))

# 因为要让对应位置
values = list(word_map_idx.values())
keys = list(word_map_idx.keys())
# word_map_idx.keys()

# with open("cluster_word_map_idx.pkl","wb") as f:
#     pickle.dump(word_map_idx,f)


In [139]:
for i in range(0,10):
    print("\n Cluster %d" % i)
    words = []
    for j in range(0, len(values)):
        if(values[j] == i):
            words.append(keys[j])
    print(words)

# max(idx)


 Cluster 0
['dude']

 Cluster 1
['whilst', 'semi', 'wacky', 'vaguely', 'riot', 'mouthed', 'delightfully', 'suitably', 'hysterically', 'deliciously', 'natured', 'darkly', 'moderately', 'puns', 'wickedly']

 Cluster 2
['spielberg', 'boll', 'fulci', 'hooper', 'uwe', 'tobe']

 Cluster 3
['billed', 'billing']

 Cluster 4
['lie', 'challenge', 'demand', 'refuse', 'address', 'expose', 'alter', 'encourage', 'possess', 'promote', 'exploit', 'sane', 'pursue', 'embrace', 'confuse', 'resolve', 'examine', 'observe', 'invite', 'fulfill', 'acknowledge', 'inform', 'manipulate', 'submit', 'educate', 'preserve']

 Cluster 5
['initially', 'ironically', 'loyal', 'andre', 'reluctant', 'baron', 'bishop', 'wang', 'mentor', 'guardian', 'christy', 'gandhi', 'ira', 'dixon', 'scheming', 'client', 'commanding', 'psychologist', 'peterson', 'jed', 'alonzo', 'maris', 'jude', 'marcel', 'mitch', 'mantle', 'antwone', 'widower']

 Cluster 6
['frightening', 'horrific', 'gruesome', 'terrifying', 'horrifying']

 Cluster 7


In [145]:
# 上面的词汇到对应聚类的映射，不直接将单词或者对应聚类变成集合的原因，是因为需要顺序地遍历，将对应聚类的单词放一起然后暂时展示出来。

# 这样的BOW范式我猜大概率更适合随机森林，随机森林基于决策树，决策树认为特征向量分量的语义特征越鲜明越好。基于频次的BOW的分量语义
# 信息其实比较微妙，作为更常出现的单词，它的语义特征到底是更鲜明还是信息熵更大其实真的比较难讲；词向量将单词映射到一个连续特征空间中
# ，在这个特征空间中，语义特征是连续的，与随机森林中离散的信息表达感觉上也并非很契合；利用词向量在特征空间中语义特征的连续性，
# 将语义特征相近的单词聚类。在这个基础上计频生成的特征向量，有理由猜想或者认为，其每一个分量——至少内部——信息熵会比较低

def bags_of_cluster(review_words, word_map_idx):
    num_clusters = max(word_map_idx.values())
    
    bags_word = np.zeros(num_clusters, dtype = 'float32')
    
    for word in review_words:
        if word in word_map_idx:
            bags_word[word_map_idx['word']] += 1
    
    return bags_word

In [146]:
# clear_train_reviews

def review_2_clusterVec(reviews, word_map_idx):
    num_cluster = max(word_map_idx.values())
    
    clusterVec = np.zeros((len(reviews),num_cluster), dtype = 'float32')

    for i in range(0, len(reviews)):
        if (i + 1) % 500 == 0:
            print("Review %d of %d" % (i+1, len(reviews)))
        clusterVec[i] = bags_of_cluster(reviews[i],word_map_idx)

    return clusterVec
    

In [147]:
train_clusterVec = review_2_clusterVec(clear_train_reviews,word_map_idx)
test_clusterVec = review_2_clusterVec(clear_test_reviews,word_map_idx)

Review 500 of 25000
Review 1000 of 25000
Review 1500 of 25000
Review 2000 of 25000
Review 2500 of 25000
Review 3000 of 25000
Review 3500 of 25000
Review 4000 of 25000
Review 4500 of 25000
Review 5000 of 25000
Review 5500 of 25000
Review 6000 of 25000
Review 6500 of 25000
Review 7000 of 25000
Review 7500 of 25000
Review 8000 of 25000
Review 8500 of 25000
Review 9000 of 25000
Review 9500 of 25000
Review 10000 of 25000
Review 10500 of 25000
Review 11000 of 25000
Review 11500 of 25000
Review 12000 of 25000
Review 12500 of 25000
Review 13000 of 25000
Review 13500 of 25000
Review 14000 of 25000
Review 14500 of 25000
Review 15000 of 25000
Review 15500 of 25000
Review 16000 of 25000
Review 16500 of 25000
Review 17000 of 25000
Review 17500 of 25000
Review 18000 of 25000
Review 18500 of 25000
Review 19000 of 25000
Review 19500 of 25000
Review 20000 of 25000
Review 20500 of 25000
Review 21000 of 25000
Review 21500 of 25000
Review 22000 of 25000
Review 22500 of 25000
Review 23000 of 25000
Review 2

In [148]:
# test_clusterVec.shape
sum(sum(train_clusterVec))

2802271.0

In [159]:
forest_2 = RandomForestClassifier(n_estimators = 100)

# forest_2.fit(train_clusterVec, labeled_data['sentiment'])

# joblib.dump(forest_2, 'kmean_w2v_forest.joblib')

forest_2 = joblib.load('kmean_w2v_forest.joblib')
result_2 = forest_2.predict(test_clusterVec)

kmean_w2v_prediction_sub = pd.DataFrame(data = {'id':test_data['id'],'sentiment':result})
kmean_w2v_prediction = pd.DataFrame(data = {'review':test_data['review'],'sentiment':result})

kmean_w2v_prediction_sub.to_csv('kmean_w2v_prediction_sub.csv',index=False,quoting = 3)
kmean_w2v_prediction.to_csv('kmean_w2v_prediction.csv',index=False)

### 结
虽然分析了一大堆，但真相是这三种方法在随机森林上的效果都差不多。
另外，词向量的平均和聚类丢失了词序信息（怎么说？）
据说有有种叫做段落向量的更好的处理方法。